---
title: "Tutorial: How to fine tune a Cellpose model"
author: Fernando Cervantes (fernando.cervantes@jax.org)
format:
  revealjs:
    code-fold: false
    progress: true
    controls: true
    fontsize: 22pt

jupyter: python3
---


# 1 Image groups management

## 1.1 Load a sample image

You can use the cells 3D image sample from the napari's built-in samples.

```
File > Open Sample > napari builtins > Cells (3D+2Ch)
```


In [ ]:
#| echo: false
import napari
from napari.utils import nbscreenshot
from PIL import Image, ImageDraw, ImageFont
import napari_activelearning as al
import zarr

In [ ]:
#| echo: false
viewer = napari.Viewer()
_ = viewer.open_sample(plugin="napari", sample="cells3d")

In [ ]:
#| echo: false
nbscreenshot(viewer)

## 1.2 Add the _Image Groups Manager_ widget to napari's window

You can find the _Image group manager_ under the _Active Learning_ plugin in the napari's plugins menu.

```
Plugins > Active Learning > Image groups manager
```


In [ ]:
#| echo: false
image_groups_mgr, acquisition_fun_cfg, labels_mgr = al.get_active_learning_widget()

In [ ]:
#| echo: false
image_groups_mgr_dw = viewer.window.add_dock_widget(image_groups_mgr)

In [ ]:
#| echo: false
nbscreenshot(viewer)

## 1.3 Create an _Image Group_ containing _nuclei_ and _membrane_ layers

- Select the _nuclei_ and _membrane_ layer and click the _New Image Group_ button on the _Image Groups Manager_ widget.


In [ ]:
#| echo: false
viewer.layers.selection.clear()
viewer.layers.selection.add(viewer.layers["nuclei"])
viewer.layers.selection.add(viewer.layers["membrane"])
image_groups_mgr.create_group()

In [ ]:
#| echo: false
nbscreenshot(viewer)

## 1.4 Edit the image group properties

:::: {.columns}

::: {.column width=0.3}
- Select the newly created image group, it will appear as "images" in the _Image groups manager_ widget.

- Click the _Edit group properties_ checkbox.

- Make sure that _Axes order_ is "CZYX", otherwise, you can edit it and press _Enter_ to update the axes names.
:::

::: {.column width=0.7}

In [ ]:
#| echo: false
image_groups_mgr.image_groups_editor._show_editor(True)
image_groups_mgr._active_image_group.child(0).setSelected(True)
image_groups_mgr.image_groups_editor.edit_axes_le.setText("CZYX")
image_groups_mgr.image_groups_editor.update_source_axes()

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = screenshot.shape[:2]

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([70, 250, 310, 280], outline="white", width=5)
draw.rectangle([70, 250, 310, 280], outline="green", width=2)

cropped_image

:::

::::

# 2 Segment the managed image groups

## 2.1 Add the _Acquisition function configuration_ widget to napari's window

The _Acquisition function configuration_ is under the _Active Learning_ plugin in the napari's plugins menu.

```
Plugins > Active Learning > Acquisition function configuration
```


In [ ]:
#| echo: false
acquisition_fun_cfg_dw = viewer.window.add_dock_widget(acquisition_fun_cfg)
viewer.window._qt_window.tabifyDockWidget(image_groups_mgr_dw, acquisition_fun_cfg_dw)

image_groups_mgr_dw.setWindowTitle("Image groups manager")
acquisition_fun_cfg_dw.setWindowTitle("Acquisition function configuration")

In [ ]:
#| echo: false
acquisition_fun_cfg_dw.raise_()

In [ ]:
#| echo: false
nbscreenshot(viewer)

## 2.2 Define sampling configuration

:::: {.columns}

::: {.column width=0.3}
### 2.2.1 Set the axes of the sampling space

1. Make sure "Input axes" are set to "ZYX"

::: {.callout-note}
This specifies that the samples will be taken from those axes.
:::

2. Change the "Model axes" to "CYX"

:::

::: {.column width=0.3}


In [ ]:
#| echo: false
acquisition_fun_cfg.input_axes_le.setText("ZYX")
acquisition_fun_cfg._set_input_axes()
acquisition_fun_cfg.model_axes_le.setText("CYX")
acquisition_fun_cfg._set_model_axes()

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = screenshot.shape[:2]

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

draw.rectangle([60, 390, 405, 425], outline="white", width=5)
draw.rectangle([60, 390, 405, 425], outline="green", width=2)

position = (250, 345)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "1", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "1", fill="white", font=font)

draw.rectangle([405, 390, 570, 425], outline="white", width=5)
draw.rectangle([405, 390, 570, 425], outline="green", width=2)

position = (490, 345)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "2", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "2", fill="white", font=font)

cropped_image

:::

::::

---

### 2.2.2 Set the size of the sampling patch

:::: {.columns}

::: {.column width=0.70}


In [ ]:
#| echo: false
acquisition_fun_cfg._show_patch_sizes(True)

acquisition_fun_cfg.patch_sizes_mspn.sizes = {"Z": 1, "Y": 256, "X": 256}

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = screenshot.shape[:2]

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([70, 240, 470, 600], outline="white", width=5)
draw.rectangle([70, 240, 470, 600], outline="green", width=2)

cropped_image

:::

::: {.column width=0.30}

- Click the "Edit patch size" checkbox
- Change the patch size of "X" and "Y" to 256, and the "Z" axis to 1.

:::{.callout-note}
This directs the Active Learning plugin to sample at random patches of size $256\times256$ pixels, and $1$ slice of depth.
:::

:::

::::

## 2.3 Define the maximum number of samples to extract

:::: {.columns}

::: {.column width=0.7}


In [ ]:
#| echo: false
acquisition_fun_cfg.max_samples_spn.setValue(4)

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = screenshot.shape[:2]

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([60, 740, 405, 770], outline="white", width=5)
draw.rectangle([60, 740, 405, 770], outline="green", width=2)

cropped_image.resize((int(0.15 * org_width), int(org_height * 0.5)), Image.Resampling.LANCZOS)

:::

::: {.column width=0.30}
- Set the "Maximum samples" to $4$ and press _Enter_

:::{.callout-note}
This tells the Active Learning plugin to process at most _four_ samples at random from the whole image.
:::

:::

::::


## 2.4 Configure the segmentation method

:::: {.columns}

::: {.column width=0.3}

1. Use the "Model" dropdown to select the `cellpose` method

2. Click the "Advanced segmentation parameters" checkbox

3. Change the "Channel axis" to $0$
::: {.callout-note}
This makes `cellpose` to use the first axis as "Color" channel.
:::

4. Change the second channel to $1$ (the right spin box in the "channels" row)

::: {.callout-note}
This tells `cellpose` to segment the first channel ($0$) and use the second channel ($1$) as help channel.
:::

5. Choose the "nuclei" model from the dropdown


:::

::: {.column width=0.7}

In [ ]:
#| echo: false
acquisition_fun_cfg.methods_cmb.setCurrentIndex(2)
acquisition_fun_cfg.tunable_segmentation_method.advanced_segmentation_options_chk.setChecked(True)
acquisition_fun_cfg.tunable_segmentation_method._segmentation_parameters.channel_axis.value = 0
acquisition_fun_cfg.tunable_segmentation_method._segmentation_parameters.channels.value = (0, 1)
acquisition_fun_cfg.tunable_segmentation_method._segmentation_parameters.model_type.value = "nuclei"

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = screenshot.shape[:2]

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([55, 490, 570, 520], outline="white", width=5)
draw.rectangle([55, 490, 570, 520], outline="green", width=2)

position = (550, 450)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "1", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "1", fill="white", font=font)

draw.rectangle([65, 525, 310, 560], outline="white", width=5)
draw.rectangle([65, 525, 310, 560], outline="green", width=2)

position = (320, 525)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "2", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "2", fill="white", font=font)

draw.rectangle([390, 655, 550, 690], outline="white", width=5)
draw.rectangle([390, 655, 550, 690], outline="green", width=2)

position = (400, 545)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "3", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "3", fill="white", font=font)

draw.rectangle([235, 580, 550, 615], outline="white", width=5)
draw.rectangle([235, 580, 550, 615], outline="green", width=2)

position = (450, 615)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "4", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "4", fill="white", font=font)

draw.rectangle([70, 725, 550, 765], outline="white", width=5)
draw.rectangle([70, 725, 550, 765], outline="green", width=2)

position = (450, 770)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "5", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "5", fill="white", font=font)

cropped_image.resize((int(0.15 * org_width), int(org_height * 0.5)), Image.Resampling.LANCZOS)

:::

::::

## 2.5 Execute the segmentation method on all image groups

:::: {.columns}

::: {.column width=0.3}
- Click the "Run on all image groups"

::: {.callout-note}
To execute the segmentation only on specific image groups, select the desired image groups in the _Image groups manager_ widget and use the "Run on selected image groups" button instead.
:::

:::

::: {.column width=0.7}

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = screenshot.shape[:2]

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([245, 850, 405, 880], outline="white", width=5)
draw.rectangle([245, 850, 405, 880], outline="green", width=2)

cropped_image.resize((int(0.15 * org_width), int(org_height * 0.5)), Image.Resampling.LANCZOS)

In [ ]:
#| echo: false
_ = acquisition_fun_cfg.compute_acquisition_layers(run_all=True)

:::

::::

## 2.6 Inspect the segmentation layer

::: {.callout-note}
Because the input image is 3D, you might have to slide the Z index on the bottom of napari's window to look at the samples that have been segmented.
:::


In [ ]:
#| echo: false
labels_mgr.focus_region(
  labels_mgr.labels_group_root.child(0).child(0)
)

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

draw = ImageDraw.Draw(image)

# Draw a red rectangle
draw.rectangle([295, 1080, 1380, 1110], outline="white", width=5)
draw.rectangle([295, 1080, 1380, 1115], outline="green", width=2)

image

# 3 Segment masked regions only

## 3.1 Create a mask to restrict the sampling space

### 3.1.1 Add a mask layer to the image group

:::: {.columns}
::: {.column width=0.3}
- Switch to the "Image groups manager" tab
- Click the "Edit mask properties" checkbox
:::

::: {.column width=0.7}

In [ ]:
#| echo: false
image_groups_mgr_dw.raise_()

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = screenshot.shape[:2]

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([65, 380, 225, 415], outline="white", width=5)
draw.rectangle([65, 380, 225, 415], outline="green", width=2)

cropped_image

In [ ]:
image_groups_mgr.mask_generator._show_editor(True)

:::

::::

---

### 3.1.2 Create a low resolution mask for its corresponding image

:::: {.columns}
:::{.column width=0.3}
1. Set the mask scale to $256$ for the "X" and "Y" axes, and a scale of $1$ for the "Z" axis

2. Click the "Create mask" button

:::{.callout-note}
This creates a low-resolution mask where each pixel corresponds to a $256\times256$ pixels region in the input image.
Because the mask is low-resolution, it uses less space (in memory RAM and disk).
:::

:::

:::{.column width=0.7}

In [ ]:
#| echo: false
image_groups_mgr.mask_generator.patch_sizes_mspn.sizes = {"Z": 1, "Y": 256, "X": 256}
image_groups_mgr.mask_generator.generate_mask_layer()

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = screenshot.shape[:2]

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([60, 450, 550, 600], outline="white", width=5)
draw.rectangle([60, 450, 550, 600], outline="green", width=2)

position = (500, 400)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "1", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "1", fill="white", font=font)

draw.rectangle([70, 620, 560, 650], outline="white", width=5)
draw.rectangle([70, 620, 560, 650], outline="green", width=2)

position = (530, 650)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "2", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "2", fill="white", font=font)

cropped_image

:::
::::


## 3.1.3 Specify the samplable regions

- Draw a mask on slices $27$ to $30$ in the "Z" axis.

:::{.callout-note}
You can move the slider at the bottom of napari's window to navigate between slices in the "Z" axis.
:::


In [ ]:
#| echo: false
viewer.camera.center = (27, 128, 128)
viewer.dims.current_step = (27, 128, 128)

viewer.layers["images mask"].data[0, 0, 27:31, 0, 0] = 1
viewer.layers["images mask"].refresh()

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

image

## 3.2 Execute the segmentation process on the masked regions

:::: {.columns}
::: {.column width=0.3}
- Go back to the "Acquisition function configuration" widget
- Click the "Run on all image groups" button again

::: {.callout-note}
Because the image group has a defined mask, samples will be extracted at random inside those defined regions only.
:::

:::

::: {.column width=0.7}

In [ ]:
#| echo: false
acquisition_fun_cfg_dw.raise_()

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = screenshot.shape[:2]

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([245, 850, 400, 890], outline="white", width=5)
draw.rectangle([245, 850, 400, 890], outline="green", width=2)

cropped_image.resize((int(0.15 * org_width), int(org_height * 0.5)), Image.Resampling.LANCZOS)

In [ ]:
#| echo: false
_ = acquisition_fun_cfg.compute_acquisition_layers(run_all=True)

:::

::::

## 3.3 Inspect the masked segmentation output


In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)
image

# 4. Fine tune the segmentation model

## 4.1 Add the _Label groups manager_ widget to napari's window

You can find the _Label groups manager_ under the _Active Learning_ plugin in the napari's plugins menu.

```
Plugins > Active Learning > Label groups manager
```


In [ ]:
#| echo: false
labels_mgr_dw = viewer.window.add_dock_widget(labels_mgr)
viewer.window._qt_window.tabifyDockWidget(acquisition_fun_cfg_dw, labels_mgr_dw)

labels_mgr_dw.setWindowTitle("Label groups manager")

In [ ]:
#| echo: false
labels_mgr_dw.raise_()

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)
image

## 4.2 Edit segmented patches

### 4.2.1. Select a segmented patch to edit

- You can double click on any segmented patch in the viewer 
(e.g. on slice $27$)


In [ ]:
#| echo: false
labels_mgr.labels_table_tw.setCurrentItem(labels_mgr.labels_group_root.child(1).child(3), 0)
labels_mgr.focus_region(
  labels_mgr.labels_group_root.child(1).child(3),
  edit_focused_label=True
)

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)
image

---

### 4.2.2. Use napari's layer controls to make changes on the objects of the current patch


In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)
image


draw = ImageDraw.Draw(image)

# Draw a red rectangle
draw.rectangle([5, 20, 280, 450], outline="white", width=5)
draw.rectangle([5, 20, 280, 450], outline="green", width=2)

image

---

### 4.2.3. Commit changes to the labels layer

:::: {.columns}
::: {.column width=0.3}

- Once you have finished editing the labels, click the "Commit changes" button on the _Label groups manager_
:::

::: {.column width=0.7}

In [ ]:
#| echo: false
z_tmp = zarr.open(r"C:\Users\cervaf\Documents\Logging\activelearning_logs\membrane.zarr\segmentation\0", mode="r")

In [ ]:
#| echo: false
labels_mgr.labels_group_root.child(1).child(3)._position
viewer.layers["Labels edit"].data[:] = z_tmp[labels_mgr.labels_group_root.child(1).child(3)._position]
labels_mgr.commit()

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = screenshot.shape[:2]

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([305, 900, 570, 940], outline="white", width=5)
draw.rectangle([305, 900, 570, 940], outline="green", width=2)

cropped_image.resize((int(0.15 * org_width), int(org_height * 0.5)), Image.Resampling.LANCZOS)

:::
::::

## 4.3 Navigate between segmented patches

:::: {.columns}
::: {.column width=0.3}

1. Expand the second group of labels 

2. Double-click on any of the nested items to open it for editing

3. Use the navigation buttons to move between segmented patches

4. Continue editing the segmentation in the current patch and commit the changes when finish
:::

::: {.column width=0.7}


In [ ]:
#| echo: false
labels_mgr.labels_table_tw.expandItem(labels_mgr.labels_group_root.child(1))
labels_mgr.labels_table_tw.setCurrentItem(labels_mgr.labels_group_root.child(1).child(0), 0)
labels_mgr.focus_region(
  labels_mgr.labels_group_root.child(1).child(0),
  edit_focused_label=True
)

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = image.height, image.width

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([70, 115, 270, 140], outline="white", width=5)
draw.rectangle([70, 115, 270, 140], outline="green", width=2)

position = (50, 105)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "1", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "1", fill="white", font=font)

draw.rectangle([110, 135, 560, 215], outline="white", width=5)
draw.rectangle([110, 135, 560, 215], outline="green", width=2)

position = (80, 150)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "2", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "2", fill="white", font=font)

draw.rectangle([60, 870, 570, 905], outline="white", width=5)
draw.rectangle([60, 870, 570, 905], outline="green", width=2)

position = (290, 820)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "3", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "3", fill="white", font=font)

# Draw a red rectangle
draw.rectangle([305, 900, 570, 940], outline="white", width=5)
draw.rectangle([305, 900, 570, 940], outline="green", width=2)

position = (270, 905)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "4", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "4", fill="white", font=font)

cropped_image.resize((int(0.15 * org_width), int(org_height * 0.5)), Image.Resampling.LANCZOS)

In [ ]:
#| echo: false
for c_idx in range(labels_mgr.labels_group_root.child(1).childCount()):
    labels_mgr.focus_region(
        labels_mgr.labels_group_root.child(1).child(c_idx),
        edit_focused_label=True
    )
    labels_mgr.labels_group_root.child(1).child(c_idx)._position
    viewer.layers["Labels edit"].data[:] = z_tmp[labels_mgr.labels_group_root.child(1).child(c_idx)._position]
    labels_mgr.commit()

:::

::::

## 4.4 Setup fine tuning configuration

### 4.4.1 Use the _Acquisition function configuration_ widget to set the configuration for executing the fine tuning process

1. Go to the "Acquisition function configuration" widget

2. Click the "Advanced fine tuning parameters" checkbox

3. Change the "save path" to a location where you want to store the fine tuned model


In [ ]:
#| echo: false
acquisition_fun_cfg_dw.raise_()

In [ ]:
#| echo: false
acquisition_fun_cfg.tunable_segmentation_method.advanced_finetuning_options_chk.setChecked(True)
acquisition_fun_cfg.tunable_segmentation_method._finetuning_parameters.save_path.value = "C:/Users/Public/Documents/models"

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = image.height, image.width

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([185, 965, 385, 1000], outline="white", width=5)
draw.rectangle([185, 965, 385, 1000], outline="green", width=2)

position = (200, 920)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "1", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "1", fill="white", font=font)

draw.rectangle([75, 690, 540, 725], outline="white", width=5)
draw.rectangle([75, 690, 540, 725], outline="green", width=2)

position = (40, 490)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "2", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "2", fill="white", font=font)

draw.rectangle([65, 495, 300, 535], outline="white", width=5)
draw.rectangle([65, 495, 300, 535], outline="green", width=2)

position = (50, 690)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "3", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "3", fill="white", font=font)

cropped_image.resize((int(0.15 * org_width), int(org_height * 0.5)), Image.Resampling.LANCZOS)

---

### 4.4.2 Set the *learning rate* and *batch size*

1. Scroll the _Advanced fine tuning parameters_ widget down to show more parameters

3. Change the "model name" to "nuclei_ft"

3. Set the "batch size" to $3$

4. Change the "learning rate" to $0.0001$

::: {.callout-note}
You can modify other parameters for the training process here, such as the number of training epochs.
:::


In [ ]:
#| echo: false
vertical_scroll_bar = acquisition_fun_cfg.tunable_segmentation_method._finetuning_parameters_scr.verticalScrollBar()
vertical_scroll_bar.setValue(vertical_scroll_bar.maximum())
 
acquisition_fun_cfg.tunable_segmentation_method._finetuning_parameters.model_name.value = "nuclei_ft"
acquisition_fun_cfg.tunable_segmentation_method._finetuning_parameters.batch_size.value = 3
acquisition_fun_cfg.tunable_segmentation_method._finetuning_parameters.learning_rate.value = 0.0001

In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = image.height, image.width

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([535, 525, 565, 845], outline="white", width=5)
draw.rectangle([535, 525, 565, 845], outline="green", width=2)

position = (515, 480)
font = ImageFont.truetype("arialbd.ttf", size=36)
draw.text(position, "1", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=36)
draw.text(position, "1", fill="white", font=font)

draw.rectangle([50, 725, 540, 755], outline="white", width=5)
draw.rectangle([50, 725, 540, 755], outline="green", width=2)

position = (20, 720)
font = ImageFont.truetype("arialbd.ttf", size=34)
draw.text(position, "2", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=34)
draw.text(position, "2", fill="white", font=font)

draw.rectangle([50, 755, 540, 780], outline="white", width=5)
draw.rectangle([50, 755, 540, 780], outline="green", width=2)

position = (20, 750)
font = ImageFont.truetype("arialbd.ttf", size=34)
draw.text(position, "3", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=34)
draw.text(position, "3", fill="white", font=font)

draw.rectangle([50, 780, 540, 810], outline="white", width=5)
draw.rectangle([50, 780, 540, 810], outline="green", width=2)

position = (20, 780)
font = ImageFont.truetype("arialbd.ttf", size=34)
draw.text(position, "4", fill="green", font=font)
font = ImageFont.truetype("arial.ttf", size=34)
draw.text(position, "4", fill="white", font=font)

cropped_image.resize((int(0.15 * org_width), int(org_height * 0.5)), Image.Resampling.LANCZOS)

## 4.5 Execute the fine tuning process

- Click the "Fine tune model" button to run the training process.

::: {.callout-note}
Depending on your computer resources (RAM, CPU), this process might take some minutes to complete. If you have a dedicated GPU device, this can take a couple of seconds instead.
:::


In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)

image = Image.fromarray(screenshot)

org_height, org_width = image.height, image.width

roi = (org_width * 0.7, 0, org_width, org_height)

# Crop the image
cropped_image = image.crop(roi)

draw = ImageDraw.Draw(cropped_image)

# Draw a red rectangle
draw.rectangle([245, 880, 405, 920], outline="white", width=5)
draw.rectangle([245, 880, 405, 920], outline="green", width=2)

cropped_image.resize((int(0.15 * org_width), int(org_height * 0.5)), Image.Resampling.LANCZOS)

In [ ]:
#| echo: false
acquisition_fun_cfg.fine_tune()

## 4.6 Review the fine tuned segmentation


In [ ]:
#| echo: false
screenshot = viewer.screenshot(canvas_only=False, flash=False)
image = Image.fromarray(screenshot)
image